In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import FodorsZagatsBenchmark

benchmark = FodorsZagatsBenchmark(data_dir_path="../data/")
benchmark

16:58:27 INFO:Extracting Fodors-Zagats...
16:58:27 INFO:Reading Fodors-Zagats row_dict...
16:58:27 INFO:Reading Fodors-Zagats train.csv...
16:58:27 INFO:Reading Fodors-Zagats valid.csv...
16:58:27 INFO:Reading Fodors-Zagats test.csv...


<FodorsZagatsBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Fodors-Zagats/fodors_zagat_exp_data.zip

## Preprocess

In [5]:
attr_list = ['name', 'addr', 'city', 'phone', 'type']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

### Rename attr `type` to `type_` (avoids clash on pytorch)

In [7]:
for row in tqdm(benchmark.row_dict.values()):
    row['type_'] = row['type']
    del row['type']

## Init Data Module

In [8]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [9]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [10]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'addr': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'city': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'phone': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'type_': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
}

In [11]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

16:58:27 INFO:For attr=name, computing actual max_str_len
16:58:27 INFO:For attr=name, using actual_max_str_len=12
16:58:27 INFO:For attr=addr, computing actual max_str_len
16:58:27 INFO:actual_max_str_len=13 must be pair to enable NN pooling. Updating to 14
16:58:27 INFO:For attr=addr, using actual_max_str_len=14
16:58:27 INFO:For attr=city, computing actual max_str_len
16:58:27 INFO:For attr=city, using actual_max_str_len=10
16:58:27 INFO:For attr=phone, computing actual max_str_len
16:58:27 INFO:For attr=phone, using actual_max_str_len=20
16:58:27 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt


{'name': NumericalizeInfo(source_attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=12, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'addr': NumericalizeInfo(source_attr='addr', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '

In [12]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [13]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [14]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

16:58:30 INFO:GPU available: True, used: True
16:58:30 INFO:TPU available: None, using: 0 TPU cores
16:58:30 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [15]:
trainer.fit(model, datamodule)

16:58:32 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 6.3 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
6.3 M     Trainable params
32.7 K    Non-trainable params
6.3 M     Total params


1

In [16]:
model.blocker_net.get_signature_weights()

{'name': 0.2226811647415161,
 'addr': 0.21684537827968597,
 'city': 0.17592975497245789,
 'phone': 0.21102944016456604,
 'type_': 0.173514261841774}

In [17]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.7586206896551725,
 'valid_f1_at_0.5': 0.9777777777777777,
 'valid_f1_at_0.7': 0.9767441860465117,
 'valid_f1_at_0.9': 0.6666666666666666,
 'valid_pair_entity_ratio_at_0.3': 0.8181818181818182,
 'valid_pair_entity_ratio_at_0.5': 0.5227272727272727,
 'valid_pair_entity_ratio_at_0.7': 0.4772727272727273,
 'valid_pair_entity_ratio_at_0.9': 0.25,
 'valid_precision_at_0.3': 0.6111111111111112,
 'valid_precision_at_0.5': 0.9565217391304348,
 'valid_precision_at_0.7': 1.0,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 1.0,
 'valid_recall_at_0.7': 0.9545454545454546,
 'valid_recall_at_0.9': 0.5}

## Testing

In [18]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.5945945945945945,
  'test_f1_at_0.5': 0.9777777777777777,
  'test_f1_at_0.7': 1.0,
  'test_f1_at_0.9': 0.625,
  'test_pair_entity_ratio_at_0.3': 1.1818181818181819,
  'test_pair_entity_ratio_at_0.5': 0.5227272727272727,
  'test_pair_entity_ratio_at_0.7': 0.5,
  'test_pair_entity_ratio_at_0.9': 0.22727272727272727,
  'test_precision_at_0.3': 0.4230769230769231,
  'test_precision_at_0.5': 0.9565217391304348,
  'test_precision_at_0.7': 1.0,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 1.0,
  'test_recall_at_0.9': 0.45454545454545453}]